In [8]:
import torch
from model import SongClassifier
from transformers import RobertaTokenizer
import sklearn.metrics
import pandas
import plotly.express as px

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [10]:
# Inicializar el modelo (aquí asumo que es una clasificación binaria)
model_name = "roberta-base"
model = SongClassifier(model_name, 5)


You are using a model of type roberta to instantiate a model of type gpt2. This is not supported for all configurations of models and can yield errors.
Some weights of GPT2Model were not initialized from the model checkpoint at roberta-base and are newly initialized: ['h.10.attn.c_attn.weight', 'h.3.attn.c_attn.bias', 'h.1.mlp.c_proj.bias', 'h.11.mlp.c_fc.weight', 'h.4.attn.c_proj.weight', 'h.5.ln_2.bias', 'h.0.attn.c_attn.bias', 'h.3.mlp.c_proj.weight', 'h.4.mlp.c_proj.bias', 'h.8.mlp.c_fc.weight', 'h.0.attn.c_proj.bias', 'h.11.attn.c_proj.weight', 'h.10.attn.c_proj.bias', 'h.5.attn.c_proj.bias', 'h.4.attn.c_proj.bias', 'h.9.ln_1.weight', 'h.9.ln_2.bias', 'h.9.attn.c_proj.bias', 'h.2.attn.c_attn.bias', 'h.0.mlp.c_proj.weight', 'h.1.ln_2.bias', 'h.11.ln_2.bias', 'h.7.attn.c_proj.bias', 'h.8.attn.c_proj.weight', 'h.5.ln_2.weight', 'h.10.ln_1.weight', 'h.3.ln_1.weight', 'h.2.ln_2.weight', 'h.4.mlp.c_fc.bias', 'h.8.mlp.c_proj.bias', 'wte.weight', 'h.3.mlp.c_proj.bias', 'h.9.ln_2.weight', 

In [11]:
# Cargar los pesos
model.load_state_dict(torch.load("../output_models/best_model.pt"))

# Mover el modelo a eval mode (esto es crucial para asegurarte de que el modelo funcione en modo inferencia y no en modo entrenamiento)
model.eval()


SongClassifier(
  (gpt2): GPT2Model(
    (wte): Embedding(50265, 768)
    (wpe): Embedding(514, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (classifier): Linear(in_features=768, out_features=5, bias=True)
)

In [12]:
tokenizer = RobertaTokenizer.from_pretrained(model_name)

In [13]:
def prediction_genero(text):
    # Tokenizar el texto
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")

    with torch.no_grad():
        logits = model(**inputs)
        probs = torch.nn.functional.softmax(logits, dim=1)
        prediction = torch.argmax(probs, dim=1)

    return prediction.item()


In [14]:
texto = "hoy hallé entre los dos el pasar de una vida el pesar de mis noches el morir de mis días sin saber si aún eras mía eras mía hubo un tiempo en que yo no quería dar un brazo a torcer pero el alma entregué porque seas mía sin saber qué hacer con tu alegría y tan lejos de mí te encontré a golpe de un mirar a punto de decir que ya no estoy aquí y tan cerca de mi yo te hallé cuando busqué sobre mi hombro vacío y encontré tu compañía cuánto puede cargar tu vida sola cuánto dura una noche cuánto pesan dos vidas sin saber qué hacer si son distintas si son distintas hubo días en que tú diste tu brazo a torcer y te ahogaste en el mar de mis recuerdos sin saber nadar y tan lejos de ti yo te hallé tan solo recordar a pasos de tu risa y frente a tu mirar y tan cerca de ti me encontré al preguntar si aún estaba vivo y encontré que eras mi vida"
prediction_genero(texto)

4

In [15]:
# Crear un DataFrame con las predicciones
df_test = pandas.read_csv("../data/test.csv")
df_test["GerneroPrediction"] = df_test["normalize_2"].apply(lambda text: prediction_genero(text))
df_test


,Genero,Artista,Titulo,Cancion,normalize_2,GerneroPrediction
0,2,mar de copas,un día sin sexo,Hoy hallé entre los dos el pasar de una vida\r...,hoy hallé entre los dos el pasar de una vida e...,4
1,3,Omar Geles,Hoja En Blanco,Fue imposible sacar tu recuerdo de mi mente\r\...,fue imposible sacar tu recuerdo de mi mente fu...,4
2,2,maná,en el muelle de san blas,Ella despidió a su amor\r\nÉl partió en un bar...,ella despidió a su amor él partió en un barco ...,4
3,3,Felipe Peláez,Ella y Tú,Ya no me pidas mas mas no puedo darte\r\nQue a...,ya no me pidas mas mas no puedo darte que amar...,3
4,4,Marc Anthony,When I Dream At Night,I have been in love\r\nAnd been alone\r\nI hav...,i have been in love and been alone i have trav...,4
...,...,...,...,...,...,...
455,4,Marc Anthony,Lamento Borincano,Sale loco de contento\r\nCon su cargamento par...,sale loco de contento con su cargamento para l...,4
456,2,poncho k,la cuenca,La cuenca\r\nSe sentó la madrugada\r\nEn tus o...,la cuenca se sentó la madrugada en tus ojos de...,4
457,2,soda estero,en el borde,Ansiedad\r\nMirada ausente\r\nNo debería esper...,ansiedad mirada ausente no debería esperar no ...,4
458,1,Daddy Yankee,"La, La, La, La (feat. Baby Rasta Y Gringo)","(se paraliza el mundo)\r\nLa, la, la, la\r\nLa...",se paraliza el mundo la la la la la la la la e...,1


In [16]:
acc = sklearn.metrics.accuracy_score(y_true=df_test["Genero"], y_pred=df_test["GerneroPrediction"])
C = sklearn.metrics.confusion_matrix(y_true=df_test["Genero"], y_pred=df_test["GerneroPrediction"])

print("Performance on Testing subset:")
print("Accuracy:")
print(acc)


Performance on Testing subset:
Accuracy:
0.4


In [17]:
f1_scores_test = sklearn.metrics.f1_score(y_true=df_test["Genero"], y_pred=df_test["GerneroPrediction"], average="weighted")
f1_scores_test

0.32068621621098237

In [18]:
labels_generos = ["pop", "reggaeton", "rock", "vallenato", "salsa"]

In [19]:
fig = px.imshow(
    C,
    text_auto=".2d",
    color_continuous_scale="greens",
    x=labels_generos,
    y=labels_generos,
    aspect="auto",
)
fig.update_layout(
    title="Matriz de confusión TEST",
    autosize=False,
    template="ggplot2",
    width=600,
    height=400,
    coloraxis_showscale=False
)
fig.show()